#SoC Classification Inference

Preventing disconnect from Google Colab notebook:
Right-click on the connect button and paste the code to the console of the UI:


```
function ClickConnect(){
    console.log("Clicked on connect button");
    document.querySelector("colab-connect-button").click() 
 }
 setInterval(ClickConnect,60000)
```

# Setup

In [1]:
%pip install fastai==2.5.3 -q
%pip freeze

     |████████████████████████████████| 189 kB 14.6 MB/s 
     |████████████████████████████████| 55 kB 4.9 MB/s 
absl-py==1.0.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.2.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.11.4
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.4.0
audioread==2.1.9
autograd==1.3
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==4.1.0
blis==0.4.1
bokeh==2.3.3
Bottleneck==1.3.4
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.10
cached-property==1.5.2
cachetools==4.2.4
catalogue==1.0.0
certifi==2021.10.8
cffi==1.15.0
cftime==1.6.0
chardet==3.0.4
charset-normalizer==2.0.12
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cupy-cuda111==9.4.0
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.28
daf

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fastai.vision.all import *
import sys

In [5]:
# Load EB_ML python libraries
# The following libraries are used in this notebook and should be installed in your local machine before running this notebook.
# eb_colab_utils.py
# eb_ml_battery_lib.py
# eb_ml_utils.py

# path to load external *.py files used in this notebook
# Note: in Google Colab virtual machine you shoud copy the files in "/content" folder after BEFORE running this notebook's cell

external_python_file_path="/content"
sys.path.append(external_python_file_path)

In [8]:
from eb_ml_utils import build_data_loader,score_model
from eb_ml_colab_utils import get_root_path
from eb_ml_battery_lib import load_soc_dataset,generate_image_files_from_eis

# Experiment configuration 

In [9]:
#configuration dictionary
config ={}

# Root working folder (local or Google Drive)
# config['ROOT_DIR'] = get_root_path("batterie")
config['ROOT_DIR'] = get_root_path("batterie")  

# Folder with dataset in CSV format
#config['DATASETS_DIR'] = config['ROOT_DIR']+"/datasets"
config['DATASETS_DIR'] = config['ROOT_DIR']+"/datasets/EIS-vs-SOC-2022"

# List of SoC level into dataset
config['soc_list']=['100','090','080','070','060','050','040','030','020','010']

# Folder to store trained model
#config['MODELS_DIR'] = config['ROOT_DIR']+"/models"
config['MODELS_DIR'] = config['ROOT_DIR']+"/models"

#TODO: nel dataset di validazione alcune classi non sono rappresentate. Usare stratified splitter
config['Splitter'] = RandomSplitter(valid_pct=0.3, seed=41)
config['rePat'] = r'^.*_(\d+).png$'


#Acquisizioni dati caricare per inferenza con modello addestrato
test_battery_list=[[6],[13],[14]]

Running on COLAB
Mounted at /gdrive


## Image file generation for inference

In [11]:
# Enable/disable image dataset generation
# Image generation process consume time and computational resources. 
# You need to generate image just once test EIS data acquisition
# generate_images = False
generate_images = False

# List EIS data acquistion that need image generation
#image_dataset_to_be_generated=[6,13,14]
image_dataset_to_be_generated=[]

#GENERATE IMAGE
if(generate_images):
  for batt in image_dataset_to_be_generated:
    config['ExperimentName']= "BATTERY_"+str(batt)
    config['TEST_IMAGES_PATH']= config['ROOT_DIR']+"/PaperEIS_PRODUCTION_DATA"
    test_dataset,feature_col_names=load_soc_dataset([batt],config["soc_list"],config['DATASETS_DIR'])
    generate_image_files_from_eis(test_dataset,feature_col_names,config['TEST_IMAGES_PATH'],config['ExperimentName'],DATA_AUGMENTATION_FACTOR=1)

# Model Ensamble

Inference with model ensamble on Test Data 

Source data: battery EIS performed for SoC 100,90,80,70,60,50,40,30,20,10. Same frequnecies for each SoC

Data format:  Re{Zbat}-Im{Zbat}j

Data Acquisition:

1. Batt1/Meas1
2. Batt1/Meas2
3. Batt2/Meas1 
4. Batt2/Meas2
5. Batt3/Meas1
6. Batt3/Meas2 --> TEST
7. Batt4/Meas1
8. Batt5/Meas1
9. Batt6/Meas1 
10. Batt7/Meas1
11. Batt8/Meas1
12. Batt9/Meas1
13. Batt10/Meas1 -->TEST
14. Batt4/Meas2 -->TEST







In [13]:
experiments=['Paper_EIS_leave_one_out_1','Paper_EIS_leave_one_out_2','Paper_EIS_leave_one_out_3','Paper_EIS_leave_one_out_4','Paper_EIS_leave_one_out_5','Paper_EIS_leave_one_out_6','Paper_EIS_leave_one_out_7','Paper_EIS_leave_one_out_8','Paper_EIS_leave_one_out_9'] # Measurement 6,13 for test]
saved_weights_files= ['/gdrive/MyDrive/batterie/models/Paper_EIS_leave_one_out_1_1648917854.175632_SAVED',
                      '/gdrive/MyDrive/batterie/models/Paper_EIS_leave_one_out_2_1648918402.165991_SAVED',
                      '/gdrive/MyDrive/batterie/models/Paper_EIS_leave_one_out_3_1648919027.947993_SAVED',
                      '/gdrive/MyDrive/batterie/models/Paper_EIS_leave_one_out_4_1648919640.51975_SAVED',
                      '/gdrive/MyDrive/batterie/models/Paper_EIS_leave_one_out_5_1648920276.006414_SAVED',
                      '/gdrive/MyDrive/batterie/models/Paper_EIS_leave_one_out_6_1648920920.623353_SAVED',
                      '/gdrive/MyDrive/batterie/models/Paper_EIS_leave_one_out_7_1648921586.201785_SAVED',
                      '/gdrive/MyDrive/batterie/models/Paper_EIS_leave_one_out_8_1648922259.048984_SAVED',
                      '/gdrive/MyDrive/batterie/models/Paper_EIS_leave_one_out_9_1648922951.265163_SAVED'
                      #TODO: 6,13,14 for test
                      ]


## Inference on new data acquisition on battery 3 (EIS Data acquisition n. 6. Battery 3 measurement 2)

In [35]:
experiments_predictions = []
experiments_predictions_decoded=[]

exeperiments_targets = []

models_accuracy = []

models_top2_accuracy = []

config['TEST_IMAGES_PATH']= config['ROOT_DIR']+"/PaperEIS_PRODUCTION_DATA/BATTERY_6"
test_fnames= get_image_files(config["TEST_IMAGES_PATH"])
for experiment_index in range(0,len(experiments),1):
  print("ExperimentName: "+str(experiments[experiment_index]))
  saved_weights=saved_weights_files[experiment_index]
  config['ExperimentName'] = experiments[experiment_index]
  config['IMAGES_PATH'] = config['ROOT_DIR']+"/"+config['ExperimentName']

  # BUILD DATA LOADER
  dl=build_data_loader(config)

  # BUILD LEARNER
  learn = cnn_learner(dl, resnet18, metrics=accuracy)

  learn = learn.load(saved_weights)

  # SCORE MODEL
  #score_model(saved_weights,dl)

  
  test_dl = learn.dls.test_dl(test_fnames,with_labels=True)
  predictions, targets, decoded = learn.get_preds(dl=test_dl, with_decoded=True)
  experiments_predictions.append(predictions)
  experiments_predictions_decoded.append(decoded)
  exeperiments_targets.append(targets)
  model_accuracy= accuracy(predictions,targets)
  models_accuracy.append(model_accuracy)
  from sklearn.metrics import top_k_accuracy_score
  model_top2_accuracy= top_k_accuracy_score(targets, predictions, k=2)
  models_top2_accuracy.append(model_top2_accuracy)
  print("model_accuracy: " + str(model_accuracy))
  print("model_top2_accuracy: " + str(model_top2_accuracy))

ExperimentName: Paper_EIS_leave_one_out_1
Setting-up type transforms pipelines
Found 900 items
2 datasets of sizes 630,270
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_1/Paper_EIS_leave_one_out_1/Batt_1710_030.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_1/Paper_EIS_leave_one_out_1/Batt_1710_030.png
    applying RegexLabeller gives
      030
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(2)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(2))


Found 900 items
2 datasets of sizes 630,270
Setting up Pipeline: PILBase.create

model_accuracy: TensorBase(0.6000)
model_top2_accuracy: 0.8
ExperimentName: Paper_EIS_leave_one_out_2
Setting-up type transforms pipelines
Found 900 items
2 datasets of sizes 630,270
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_2/Paper_EIS_leave_one_out_2/Batt_1710_030.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_2/Paper_EIS_leave_one_out_2/Batt_1710_030.png
    applying RegexLabeller gives
      030
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(2)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(2))


Found 900 items
2 d

model_accuracy: TensorBase(0.6000)
model_top2_accuracy: 0.9
ExperimentName: Paper_EIS_leave_one_out_3
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_3/Paper_EIS_leave_one_out_3/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_3/Paper_EIS_leave_one_out_3/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.6000)
model_top2_accuracy: 0.9
ExperimentName: Paper_EIS_leave_one_out_4
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_4/Paper_EIS_leave_one_out_4/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_4/Paper_EIS_leave_one_out_4/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.7000)
model_top2_accuracy: 0.9
ExperimentName: Paper_EIS_leave_one_out_5
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_5/Paper_EIS_leave_one_out_5/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_5/Paper_EIS_leave_one_out_5/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.5000)
model_top2_accuracy: 0.8
ExperimentName: Paper_EIS_leave_one_out_6
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_6/Paper_EIS_leave_one_out_6/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_6/Paper_EIS_leave_one_out_6/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.6000)
model_top2_accuracy: 0.9
ExperimentName: Paper_EIS_leave_one_out_7
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_7/Paper_EIS_leave_one_out_7/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_7/Paper_EIS_leave_one_out_7/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.6000)
model_top2_accuracy: 1.0
ExperimentName: Paper_EIS_leave_one_out_8
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_8/Paper_EIS_leave_one_out_8/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_8/Paper_EIS_leave_one_out_8/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.9000)
model_top2_accuracy: 0.9
ExperimentName: Paper_EIS_leave_one_out_9
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_9/Paper_EIS_leave_one_out_9/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_9/Paper_EIS_leave_one_out_9/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.5000)
model_top2_accuracy: 0.9


In [36]:
models_accuracy

[TensorBase(0.6000),
 TensorBase(0.6000),
 TensorBase(0.6000),
 TensorBase(0.7000),
 TensorBase(0.5000),
 TensorBase(0.6000),
 TensorBase(0.6000),
 TensorBase(0.9000),
 TensorBase(0.5000)]

In [102]:
np.mean(models_accuracy)

0.6222222

In [37]:
models_top2_accuracy

[0.8, 0.9, 0.9, 0.9, 0.8, 0.9, 1.0, 0.9, 0.9]

In [103]:
np.mean(models_top2_accuracy)

0.8888888888888888

In [104]:
# Calculate ensable prediction
ensamble_prediction = experiments_predictions[0]
for pred in experiments_predictions[1:]:
  ensamble_prediction += pred

ensamble_prediction /= len(experiments_predictions)
#ensamble_accuracy
accuracy(ensamble_prediction, exeperiments_targets[0])

TensorBase(0.6000)

## Inference on new battery (EIS data acquistion n. 13. Battery 10 - Measurement 1)

In [14]:
experiments_predictions = []

exeperiments_targets = []

models_accuracy = []

models_top2_accuracy = []

config['TEST_IMAGES_PATH']= config['ROOT_DIR']+"/PaperEIS_PRODUCTION_DATA/BATTERY_13"
test_fnames= get_image_files(config["TEST_IMAGES_PATH"])
for experiment_index in range(0,len(experiments),1):
  print("ExperimentName: "+str(experiments[experiment_index]))
  saved_weights=saved_weights_files[experiment_index]
  config['ExperimentName'] = experiments[experiment_index]
  config['IMAGES_PATH'] = config['ROOT_DIR']+"/"+config['ExperimentName']

  # BUILD DATA LOADER
  dl=build_data_loader(config)

  # BUILD LEARNER
  learn = cnn_learner(dl, resnet18, metrics=accuracy)

  learn = learn.load(saved_weights)

  # SCORE MODEL
  #score_model(saved_weights,dl)

  
  test_dl = learn.dls.test_dl(test_fnames,with_labels=True)
  predictions, targets, decoded = learn.get_preds(dl=test_dl, with_decoded=True)
  experiments_predictions.append(predictions)
  exeperiments_targets.append(targets)
  model_accuracy= accuracy(predictions,targets)
  models_accuracy.append(model_accuracy)
  from sklearn.metrics import top_k_accuracy_score
  model_top2_accuracy= top_k_accuracy_score(targets, predictions, k=2)
  models_top2_accuracy.append(model_top2_accuracy)
  print("model_accuracy: " + str(model_accuracy))
  print("model_top2_accuracy: " + str(model_top2_accuracy))

ExperimentName: Paper_EIS_leave_one_out_1
Setting-up type transforms pipelines
Found 900 items
2 datasets of sizes 630,270
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_1/Paper_EIS_leave_one_out_1/Batt_1710_030.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_1/Paper_EIS_leave_one_out_1/Batt_1710_030.png
    applying RegexLabeller gives
      030
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(2)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(2))


Found 900 items
2 datasets of sizes 630,270
Setting up Pipeline: PILBase.create

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

model_accuracy: TensorBase(0.8000)
model_top2_accuracy: 0.8
ExperimentName: Paper_EIS_leave_one_out_2
Setting-up type transforms pipelines
Found 900 items
2 datasets of sizes 630,270
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_2/Paper_EIS_leave_one_out_2/Batt_1710_030.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_2/Paper_EIS_leave_one_out_2/Batt_1710_030.png
    applying RegexLabeller gives
      030
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(2)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(2))


Found 900 items
2 d

model_accuracy: TensorBase(0.7000)
model_top2_accuracy: 1.0
ExperimentName: Paper_EIS_leave_one_out_3
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_3/Paper_EIS_leave_one_out_3/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_3/Paper_EIS_leave_one_out_3/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

...

model_accuracy: TensorBase(0.8000)
model_top2_accuracy: 0.9
ExperimentName: Paper_EIS_leave_one_out_6
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_6/Paper_EIS_leave_one_out_6/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_6/Paper_EIS_leave_one_out_6/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.7000)
model_top2_accuracy: 1.0
ExperimentName: Paper_EIS_leave_one_out_7
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_7/Paper_EIS_leave_one_out_7/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_7/Paper_EIS_leave_one_out_7/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.5000)
model_top2_accuracy: 0.8
ExperimentName: Paper_EIS_leave_one_out_8
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_8/Paper_EIS_leave_one_out_8/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_8/Paper_EIS_leave_one_out_8/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.8000)
model_top2_accuracy: 1.0
ExperimentName: Paper_EIS_leave_one_out_9
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_9/Paper_EIS_leave_one_out_9/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_9/Paper_EIS_leave_one_out_9/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.7000)
model_top2_accuracy: 1.0


In [15]:
models_accuracy

[TensorBase(0.8000),
 TensorBase(0.7000),
 TensorBase(0.6000),
 TensorBase(0.7000),
 TensorBase(0.8000),
 TensorBase(0.7000),
 TensorBase(0.5000),
 TensorBase(0.8000),
 TensorBase(0.7000)]

In [108]:
np.mean(models_accuracy)

0.6222222

In [16]:
models_top2_accuracy

[0.8, 1.0, 1.0, 1.0, 0.9, 1.0, 0.8, 1.0, 1.0]

In [109]:
np.mean(models_top2_accuracy)

0.8888888888888888

In [17]:
# Calculate ensable prediction
ensamble_prediction = experiments_predictions[0]
for pred in experiments_predictions[1:]:
  ensamble_prediction += pred

In [18]:
ensamble_prediction /= len(experiments_predictions)
#ensamble_accuracy
accuracy(ensamble_prediction, exeperiments_targets[0])

TensorBase(0.7000)

In [19]:
#ensamble top2 accuracy
top_k_accuracy_score(exeperiments_targets[0], ensamble_prediction, k=2)

1.0

## Inference on new data acquisition on battery 4 (EIS Data acquisition n. 14. Battery 4 measurement 2)

In [110]:
experiments_predictions = []

exeperiments_targets = []

models_accuracy = []

models_top2_accuracy = []

config['TEST_IMAGES_PATH']= config['ROOT_DIR']+"/PaperEIS_PRODUCTION_DATA/BATTERY_14"
test_fnames= get_image_files(config["TEST_IMAGES_PATH"])
for experiment_index in range(0,len(experiments),1):
  print("ExperimentName: "+str(experiments[experiment_index]))
  saved_weights=saved_weights_files[experiment_index]
  config['ExperimentName'] = experiments[experiment_index]
  config['IMAGES_PATH'] = config['ROOT_DIR']+"/"+config['ExperimentName']

  # BUILD DATA LOADER
  dl=build_data_loader(config)

  # BUILD LEARNER
  learn = cnn_learner(dl, resnet18, metrics=accuracy)

  learn = learn.load(saved_weights)

  # SCORE MODEL
  #score_model(saved_weights,dl)

  
  test_dl = learn.dls.test_dl(test_fnames,with_labels=True)
  predictions, targets, decoded = learn.get_preds(dl=test_dl, with_decoded=True)
  experiments_predictions.append(predictions)
  exeperiments_targets.append(targets)
  model_accuracy= accuracy(predictions,targets)
  models_accuracy.append(model_accuracy)
  from sklearn.metrics import top_k_accuracy_score
  model_top2_accuracy= top_k_accuracy_score(targets, predictions, k=2)
  models_top2_accuracy.append(model_top2_accuracy)
  print("model_accuracy: " + str(model_accuracy))
  print("model_top2_accuracy: " + str(model_top2_accuracy))

ExperimentName: Paper_EIS_leave_one_out_1
Setting-up type transforms pipelines
Found 900 items
2 datasets of sizes 630,270
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_1/Paper_EIS_leave_one_out_1/Batt_1710_030.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_1/Paper_EIS_leave_one_out_1/Batt_1710_030.png
    applying RegexLabeller gives
      030
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(2)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(2))


Found 900 items
2 datasets of sizes 630,270
Setting up Pipeline: PILBase.create

model_accuracy: TensorBase(0.5000)
model_top2_accuracy: 0.7
ExperimentName: Paper_EIS_leave_one_out_2
Setting-up type transforms pipelines
Found 900 items
2 datasets of sizes 630,270
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_2/Paper_EIS_leave_one_out_2/Batt_1710_030.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_2/Paper_EIS_leave_one_out_2/Batt_1710_030.png
    applying RegexLabeller gives
      030
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(2)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(2))


Found 900 items
2 d

model_accuracy: TensorBase(0.2000)
model_top2_accuracy: 0.7
ExperimentName: Paper_EIS_leave_one_out_3
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_3/Paper_EIS_leave_one_out_3/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_3/Paper_EIS_leave_one_out_3/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.5000)
model_top2_accuracy: 0.6
ExperimentName: Paper_EIS_leave_one_out_4
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_4/Paper_EIS_leave_one_out_4/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_4/Paper_EIS_leave_one_out_4/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.4000)
model_top2_accuracy: 0.6
ExperimentName: Paper_EIS_leave_one_out_5
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_5/Paper_EIS_leave_one_out_5/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_5/Paper_EIS_leave_one_out_5/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.4000)
model_top2_accuracy: 0.8
ExperimentName: Paper_EIS_leave_one_out_6
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_6/Paper_EIS_leave_one_out_6/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_6/Paper_EIS_leave_one_out_6/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.5000)
model_top2_accuracy: 0.7
ExperimentName: Paper_EIS_leave_one_out_7
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_7/Paper_EIS_leave_one_out_7/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_7/Paper_EIS_leave_one_out_7/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.4000)
model_top2_accuracy: 0.7
ExperimentName: Paper_EIS_leave_one_out_8
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_8/Paper_EIS_leave_one_out_8/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_8/Paper_EIS_leave_one_out_8/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.6000)
model_top2_accuracy: 0.8
ExperimentName: Paper_EIS_leave_one_out_9
Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_9/Paper_EIS_leave_one_out_9/Batt_1702_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_9/Paper_EIS_leave_one_out_9/Batt_1702_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2

model_accuracy: TensorBase(0.4000)
model_top2_accuracy: 0.7


In [111]:
models_accuracy

[TensorBase(0.5000),
 TensorBase(0.2000),
 TensorBase(0.5000),
 TensorBase(0.4000),
 TensorBase(0.4000),
 TensorBase(0.5000),
 TensorBase(0.4000),
 TensorBase(0.6000),
 TensorBase(0.4000)]

In [112]:
np.mean(models_accuracy)

0.43333334

In [113]:
models_top2_accuracy

[0.7, 0.7, 0.6, 0.6, 0.8, 0.7, 0.7, 0.8, 0.7]

In [114]:
np.mean(models_top2_accuracy)

0.7

In [115]:
# Calculate ensable prediction
ensamble_prediction = experiments_predictions[0]
for pred in experiments_predictions[1:]:
  ensamble_prediction += pred

ensamble_prediction /= len(experiments_predictions)
#ensamble_accuracy
accuracy(ensamble_prediction, exeperiments_targets[0])

TensorBase(0.4000)

In [116]:
#ensamble top2 accuracy
top_k_accuracy_score(exeperiments_targets[0], ensamble_prediction, k=2)

0.7

# Single Model - Training/Validation split

Modello addestrato con tutti i dati tranne batteria 3 e 10

In [46]:
#model trained on data acquisition from batteries 1,2,4,5,6,7,8,9
# acquistion from batteries 3 and 10 used only for inference test
saved_weights = "/gdrive/MyDrive/batterie/models/Paper_EIS_single_model_1648968761.170263_SAVED"

config['ExperimentName'] = "PaperEIS_3"
config['IMAGES_PATH'] = config['ROOT_DIR']+"/"+config['ExperimentName']

## Test on new data acquisition on battery 3

In [83]:
config['TEST_IMAGES_PATH']= config['ROOT_DIR']+"/PaperEIS_PRODUCTION_DATA/BATTERY_6"
test_fnames= get_image_files(config["TEST_IMAGES_PATH"])
#DATA LOADER
dl=build_data_loader(config)

# BUILD LEARNER
learn = cnn_learner(dl, resnet18, metrics=accuracy)

learn = learn.load(saved_weights)

test_dl = learn.dls.test_dl(test_fnames,with_labels=True)
predictions, targets, decoded = learn.get_preds(dl=test_dl, with_decoded=True)
model_accuracy= accuracy(predictions,targets)
from sklearn.metrics import top_k_accuracy_score
model_top2_accuracy= top_k_accuracy_score(targets, predictions, k=2)
print("model_accuracy: " + str(model_accuracy))
print("model_top2_accuracy: " + str(model_top2_accuracy))

Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/PaperEIS_3/PaperEIS_3/Batt_1027_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/PaperEIS_3/PaperEIS_3/Batt_1027_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}


model_accuracy: TensorBase(0.8000)
model_top2_accuracy: 0.9


In [84]:
# SCORE MODEL
score_model(saved_weights,dl)

[[26  0  0  0  0  0  0  0  0  0]
 [ 0 36  0  0  0  0  0  0  0  0]
 [ 0  0 27  0  3  0  0  0  0  0]
 [ 0  3  0 33  0  0  0  0  0  0]
 [ 0  2  0  0 25  0  0  0  0  0]
 [ 0  0  0  0  0 28  0  0  0  0]
 [ 0  0  0  0  0  0 26  2  0  0]
 [ 0  0  0  0  0  0  1 22  5  0]
 [ 0  0  0  0  0  0  0  0 35  0]
 [ 0  0  0  0  0  0  0  0  0 26]]
[[1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         1.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.9        0.         0.1        0.
  0.         0.         0.         0.        ]
 [0.         0.08333333 0.         0.91666667 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.07407407 0.         0.         0.92592593 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         1.
  0.         0.         0.         0.        ]
 [0.         0.         0. 

learn.validate()


Model accuracy: 0.95
learn.get_preds()


TensorBase(0.9467)

## Test on new battery (battery 10)

In [85]:
config['TEST_IMAGES_PATH']= config['ROOT_DIR']+"/PaperEIS_PRODUCTION_DATA/BATTERY_13"
test_fnames= get_image_files(config["TEST_IMAGES_PATH"])
#DATA LOADER
dl=build_data_loader(config)

# BUILD LEARNER
learn = cnn_learner(dl, resnet18, metrics=accuracy)

learn = learn.load(saved_weights)

test_dl = learn.dls.test_dl(test_fnames,with_labels=True)
predictions, targets, decoded = learn.get_preds(dl=test_dl, with_decoded=True)
model_accuracy= accuracy(predictions,targets)
from sklearn.metrics import top_k_accuracy_score
model_top2_accuracy= top_k_accuracy_score(targets, predictions, k=2)
print("model_accuracy: " + str(model_accuracy))
print("model_top2_accuracy: " + str(model_top2_accuracy))

Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/PaperEIS_3/PaperEIS_3/Batt_1027_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/PaperEIS_3/PaperEIS_3/Batt_1027_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}


model_accuracy: TensorBase(0.6000)
model_top2_accuracy: 0.9


## Test on new battery 4 measurement (EIS Data acquisition 14, Battery 4 - measurement 2)

In [86]:
config['TEST_IMAGES_PATH']= config['ROOT_DIR']+"/PaperEIS_PRODUCTION_DATA/BATTERY_14"
test_fnames= get_image_files(config["TEST_IMAGES_PATH"])
#DATA LOADER
dl=build_data_loader(config)

# BUILD LEARNER
learn = cnn_learner(dl, resnet18, metrics=accuracy)

learn = learn.load(saved_weights)

test_dl = learn.dls.test_dl(test_fnames,with_labels=True)
predictions, targets, decoded = learn.get_preds(dl=test_dl, with_decoded=True)
model_accuracy= accuracy(predictions,targets)
from sklearn.metrics import top_k_accuracy_score
model_top2_accuracy= top_k_accuracy_score(targets, predictions, k=2)
print("model_accuracy: " + str(model_accuracy))
print("model_top2_accuracy: " + str(model_top2_accuracy))

Setting-up type transforms pipelines
Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/PaperEIS_3/PaperEIS_3/Batt_1027_010.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/PaperEIS_3/PaperEIS_3/Batt_1027_010.png
    applying RegexLabeller gives
      010
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(0)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(0))


Found 1000 items
2 datasets of sizes 700,300
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}


model_accuracy: TensorBase(0.5000)
model_top2_accuracy: 0.6


In [87]:
def get_analog_value(prediction_output,learner):
  ind = np.argsort(prediction_output)
  value1 = prediction_output[ind[len(prediction_output)-1]]
  print(value1)
  value2 = prediction_output[ind[len(prediction_output)-2]]
  print(value2)

  classIndex1= ind[len(prediction_output)-1]
  print(classIndex1)

  classIndex2= ind[len(prediction_output)-2]
  print(classIndex2)

  avg_value= (int(learner.dls.vocab[classIndex2]) * value2) + (int(learner.dls.vocab[classIndex1]) * value1)
  return avg_value

In [88]:
targets

TensorCategory([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])

In [89]:
ind=np.argsort(predictions[0])

In [90]:
ind

TensorBase([5, 7, 8, 6, 4, 2, 3, 0, 1, 9])

In [91]:
value1 = predictions[0][ind[len(predictions[0])-1]]

In [92]:
value1

TensorBase(0.7482)

In [93]:
value2 = predictions[0][ind[len(predictions[0])-2]]

In [94]:
value2

TensorBase(0.2101)

In [95]:
classIndex1= ind[len(predictions[0])-1]
print(classIndex1)

classIndex2= ind[len(predictions[0])-2]
print(classIndex2)


TensorBase(9)
TensorBase(1)


In [96]:
predictions[0]

TensorBase([3.4467e-02, 2.1010e-01, 3.5740e-04, 6.6932e-03, 1.8766e-04, 6.3845e-08,
        3.8220e-06, 9.1486e-07, 9.7353e-07, 7.4819e-01])

In [97]:
learn


In [98]:
  avg_value= (int(learn.dls.vocab[classIndex2]) * value2) + (int(learn.dls.vocab[classIndex1]) * value1)

In [99]:
avg_value

TensorBase(79.0211)

In [100]:
get_analog_value(predictions[0],learn)

TensorBase(0.7482)
TensorBase(0.2101)
TensorBase(9)
TensorBase(1)


TensorBase(79.0211)

In [101]:
get_analog_value(predictions[1],learn)

TensorBase(0.8190)
TensorBase(0.1810)
TensorBase(0)
TensorBase(1)


TensorBase(11.8096)